In [1]:
import os
import re
import pickle
from pathlib import Path
import yaml
import json

import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import torch
import tifffile
from einops import rearrange, repeat

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import mushroom.utils as utils

In [6]:
project_dir = Path('/data/estorrs/mushroom/data/projects/submission_v1')

In [7]:
# source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
# target_root = '/data/estorrs/mushroom/data'

# def alter_filesystem(config, source_root, target_root):
#     for entry in config['sections']:
#         for mapping in entry['data']:
#             mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
    
#     if 'trainer_kwargs' in config and config['trainer_kwargs']['data_mask'] is not None:
#         config['trainer_kwargs']['data_mask'] = config['trainer_kwargs']['data_mask'].replace(source_root, target_root)
        
#     return config

In [8]:
case_order = [
    'HT206B1',
    'HT268B1',
    'HT339B2-H1',
    'HT397B1',
    'HT565B1-H2',
    'HT704B1',
    'HT891Z1',
    'HT913Z1',
    'S18-5591-C8',
    'S18-9906',
]

In [ ]:
'/da